# Downloading papers using PyAlex 

In [1]:
pip install pyalex pandas


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install ace_tools


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from pyalex import Works, Authors, Sources, Institutions, Topics

In [10]:
# Test query: Fetching 5 recent papers related to AI
sample_papers = Works().search("artificial intelligence").get(5)

# Print one sample paper
print(sample_papers[0])

# Test query worked

[{'id': 'https://openalex.org/W2122410182', 'doi': 'https://doi.org/10.5860/choice.33-1577', 'title': 'Artificial intelligence: a modern approach', 'display_name': 'Artificial intelligence: a modern approach', 'relevance_score': 15064.875, 'publication_year': 1995, 'publication_date': '1995-11-01', 'ids': {'openalex': 'https://openalex.org/W2122410182', 'doi': 'https://doi.org/10.5860/choice.33-1577', 'mag': '2122410182'}, 'language': 'en', 'primary_location': {'is_oa': False, 'landing_page_url': 'https://doi.org/10.5860/choice.33-1577', 'pdf_url': None, 'source': {'id': 'https://openalex.org/S2764375719', 'display_name': 'Choice Reviews Online', 'issn_l': '0009-4978', 'issn': ['0009-4978', '1523-8253', '1943-5975'], 'is_oa': False, 'is_in_doaj': False, 'is_indexed_in_scopus': False, 'is_core': True, 'host_organization': 'https://openalex.org/P4310316146', 'host_organization_name': 'Association of College and Research Libraries', 'host_organization_lineage': ['https://openalex.org/P431

/var/folders/_4/nzq6mygj7j71_l3z_c9kc7wr0000gn/T/ipykernel_74590/702196258.py:2: DeprecationWarning: return_meta is deprecated, call .meta on the result
  sample_papers = Works().search("artificial intelligence").get(5)


Let's define our search criteria now that our test query worked. 

Topic: AI for pricing and promotion for GDO 
Date Range: Last 10 years (2014-2024)
Fields to extract:
- id: ArXiv ID
- submitter: Who submitted the paper
- authors: Authors of the paper
- title: Title of the paper
- comments: Additional info (e.g., number of pages and figures)
- Journ-ref: Information about the journal
- doi: Digital Object Identifier
- abstract: The abstract of the paper
- versions: A version history

Useful links: https://docs.openalex.org/api-entities/entities-overview and https://github.com/J535D165/pyalex

In [23]:
# Define search parameters
query = "AI AND pricing AND promotion AND GDO"
year_start = 2014
year_end = 2024

# Fetch papers from OpenAlex
papers = Works().search(query).filter(
    from_publication_date=f"{year_start}-01-01",
    to_publication_date=f"{year_end}-12-31"
).get()

# Function to reconstruct abstracts if stored in abstract_inverted_index format
def reconstruct_abstract(abstract_index):
    if not abstract_index:
        return None
    words = []
    for key, positions in sorted(abstract_index.items(), key=lambda x: min(x[1])):
        words.append(key)
    return " ".join(words)

# Extract relevant information safely
data = []
for paper in papers:
    primary_location = paper.get("primary_location", {})  # Handle missing journal info
    source = primary_location.get("source")  # Might be None
    source_name = source["display_name"] if source else "Unknown"

    data.append({
        "id": paper.get("id"),
        "title": paper.get("title"),
        "authors": ", ".join(
            [author["author"]["display_name"] for author in paper.get("authorships", []) if author.get("author")]
        ),
        "abstract": reconstruct_abstract(paper.get("abstract_inverted_index")),  # Reconstruct abstract
        "year": paper.get("publication_year"),
        "journal": source_name,  # Get journal name safely
        "keywords": paper.get("keywords", []),
        "topics": [topic["display_name"] for topic in paper.get("topics", [])],  # Extract topics
    })

# Convert to DataFrame
df = pd.DataFrame(data)


In [24]:
# Display the number of articles in the dataframe
num_articles = len(df)
num_articles

5

In [25]:
display(df)

,id,title,authors,abstract,year,journal,keywords,topics
0,https://openalex.org/W4214872553,Land administration system and geoportal servi...,"Agnieszka Dawidowicz, Magdalena Nowak, Marta G...",Motives: IT is nowadays an effective tool supp...,2022,Acta Scientiarum Polonorum Administratio Locorum,[{'id': 'https://openalex.org/keywords/geoport...,"[Land Use and Ecosystem Services, Urban Green ..."
1,https://openalex.org/W3197052919,Dynamic Hyperparameter Allocation under Time C...,"Jeongcheol Lee, Sunil Ahn, Hyunseob Kim, Jongs...",Automated hyperparameter optimization (HPO) is...,2021,Intelligent Automation & Soft Computing,[{'id': 'https://openalex.org/keywords/hyperpa...,"[Machine Learning and Data Classification, Adv..."
2,https://openalex.org/W4205699678,Enhancing the social partners and social dialo...,Lorenzo Bordogna,"Written before the Covid-19 crisis, chapter, a...",2021,Edward Elgar Publishing eBooks,[{'id': 'https://openalex.org/keywords/social-...,"[Labor Movements and Unions, Digital Economy a..."
3,https://openalex.org/W4323026858,Market Access: Nuovi paradigmi manageriali e s...,"Nicola Saraceni, Giuseppe Galliverti, Federico...",If the NHS and its healthcare companies have o...,2023,Unknown,[{'id': 'https://openalex.org/keywords/consoli...,"[Management, Economics, and Public Policy]"
4,https://openalex.org/W4280652792,The value of the partnership between the pharm...,aa.vv. aa.vv.,o Clinical research has undergone a profound t...,2022,Unknown,"[{'id': 'https://openalex.org/keywords/value',...","[Science, Research, and Medicine]"


In [ ]:
# Save to CSV
# You might need to change the path 
dionne_path = "/Users/dionnespaltman/Desktop/Luiss /Data Science in Action/Project/openalex_papers.csv"
df.to_csv(dionne_path, index=False)

print("Extracted", len(df), "papers and saved them to 'openalex_papers.csv'.")

In [22]:
# Display extracted data
import ace_tools as tools
tools.display_dataframe_to_user(name="Bulk Downloaded OpenAlex Papers", dataframe=df)

ModuleNotFoundError: No module named 'ace_tools'